In [ ]:
import os

# plotting libraries
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from toolz import valmap

# colorblind friendly defaults
import gnl
from uwnet.analysis.sam_debug import *
from uwnet.interface import step_with_xarray_inputs
from uwnet.model import MLP

hv.extension('bokeh')

gnl.colorblind()

In [ ]:
ids = {'active': 'compassionate_chandrasekhar', 'passive': 'romantic_volta'}
training_data_path = "../data/training_data.nc"
model_path = "../data/samNN/compassionate_chandrasekhar/NG1/data.pkl"


paths = valmap(lambda x: os.path.join("../data", "samNN", x), ids)


def _open_debug(t):
    ds = open_debug_and_training_data(t, paths, training_data_path)
    return t * 30 / 60, ds


training_ds = xr.open_dataset(training_data_path)
ds = concat_datasets([_open_debug(t) for t in range(20)], name='t')
model = MLP.from_dict(torch.load(model_path)['dict'])

In [ ]:
ds

Let's find a point with strong precipitation:

# Precipitation

In [ ]:
i, j = (ds.Prec.sel(tag='active')[-1] > 115).values.nonzero()
loc = ds.isel(x=j, y=i).squeeze()

In [ ]:
loc.Prec.plot(hue='tag')

We can see that the initial precipitation is way to high for this point. Why? Let's compute the neural networks prediction of precip for every point in the training data.

In [ ]:
def _compute_precip(x):
    out_ds = step_with_xarray_inputs(model.step, x, 10.0)
    return out_ds.Prec

prec = training_ds.groupby('time').apply(_compute_precip)

And plot it:

In [ ]:
x = prec.mean(['x', 'y'])
y = training_ds.Prec.mean(['x', 'y'])

scatter = hv.Scatter((x, y)) *\
hv.Curve([(x.min(), x.min()), (x.max(), x.max())])

time_series = hv.Curve(x, label="NN") * hv.Curve(y, label="NGAqua")

scatter.redim(x="NN Prec", y="NGAqua Prec") + time_series.opts(plot=dict(width=500))

It actually looks like the neural network under-estimates the mean precipitation for the time points with higher domain averaged precip. The neural network struggles with the diurnal cycle of precipitation. The time points used for the two dimensional fields are probably mis-aligned in the training data.

In [ ]:
args  = tuple(np.ravel(x) for x in 
        xr.broadcast(prec, training_ds.Prec))
datashade(hv.Scatter(args)).redim(x="NN", y="Training Data").relabel("Precipitation (mm/day)")

# Vertical Velocity

What is $W$ doing:

In [ ]:
loc.W.plot(row='tag', x='t', aspect=2)
plt.xlabel('Minutes')

We can see that this point has a very strong upward vertical velocity throughout the troposphere. In the active simulation, this vetical velocity increases dramatically within 10 minutes.

In [ ]:
loc.W.isel(z=8).plot(hue='tag')

There is a nearly linear increase in W. I would expect this to look more exponential if it were some kind of neural network instability.

In [ ]:
sli_anom = loc.SLI - loc.SLI[0]
sli_anom.plot(row='tag', x='t', aspect=2)
plt.xlabel('Minutes')

In [ ]:
qt_anom = loc.QT - loc.QT[0]
qt_anom.plot(row='tag', x='t', aspect=2)
plt.suptitle('QT anomaly from initial value (g/kg)', y=1.05)
plt.xlabel('Minutes')